# Dynamic Optimization With Bequest Taxation
Imports and set magics:

In [20]:
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import ipywidgets as widgets
import time
from scipy import linalg
from scipy import optimize
import sympy as sm
from IPython.display import display

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import modelproject

%matplotlib inline
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


To start, we consider a simple two period consumer problem without uncertainty, where the following definitions of **variables** and **parameters** apply:

* $m$ is the endowment (cash-on-hand). In this simplified version the consumer only get an endowment in period 1.
* $c$ is consumption
* $y$ is income
* $r$ is the interest rate
* $\beta > 0$, $\rho > 1$, $\alpha \in (0,1)$, $\nu > 0 $, $\kappa > 0$ are utility parameters
* Subscripts denotes periods

Furthermore, we introduce $\tau$ as the bequest taxation rate $0<\tau<1$.

The two period utility function, $U$, can then be written as 

$$
\begin{align}
    U &= \max_{c_1,c_2}\frac{c_1^{1-\rho}}{1-\rho}+\beta\frac{c_2^{1-\rho}}{1-\rho} \\
    \text{s.t.} \\
    c_2 &= (1+r)(m_1-c_1) \\
    c_{1} &\in [0,m_{1}] \\
\end{align}
$$

where the constraint on consumption in period 2 is derived from the intertemporal budget constraint. Borrowing is not possible by the second constraint. 


In [2]:
# define symbols
m1 = sm.symbols('m_1')
c1 = sm.symbols('c_1')
c2 = sm.symbols('c_2')
r = sm.symbols('r')

beta = sm.symbols('beta')
kappa = sm.symbols('kappa')
nu = sm.symbols('nu')
rho = sm.symbols('rho')

# define objective
m2 = (1+r)*(m1-c1)
obj = c1**(1-rho)/(1-rho)+beta*(c2**(1-rho)/(1-rho))

# define constraint and solve for c2
cons = sm.Eq(c2,(1+r)*(m1-c1))
c2_from_cons = sm.solve(cons,c2)

# substitute for c2 and take first order conditions
obj_subs = obj.subs(c2,c2_from_cons[0])

dU_dc1 = sm.diff(obj_subs,c1)
dU_dc1

beta*(-r - 1)*(-c_1*r - c_1 + m_1*r + m_1)**(1 - rho)/(-c_1*r - c_1 + m_1*r + m_1) + c_1**(1 - rho)/c_1

Before we can solve for optimal consumption in period 1, we need to help sympy understand that the above expression is equal to 
$$
\begin{align}
    \frac{1}{c_1} &= \frac{\left(\beta(1+r)\right)^\frac{1}{\rho}}{(1+r)(m_1-c_1)}
\end{align}
$$

In [3]:
# help sympy collect terms
dU_dc1_simple = sm.Eq(1/c1,(beta*(1+r))**(1/rho)/((1+r)*(m1-c1)))
display(dU_dc1_simple)

Eq(1/c_1, (beta*(r + 1))**(1/rho)/((-c_1 + m_1)*(r + 1)))

In [4]:
# solve for optimal c1
c1_star_solve = sm.solve(dU_dc1_simple,c1)
c1_star = c1_star_solve[0]

m_1*(r + 1)/(r + (beta*(r + 1))**(1/rho) + 1)

In [6]:
# compute optimal c2
cons_subs = cons.subs(c1,c1_star[0])
c2_star_solve = sm.solve(cons_subs,c2)
c2_star = c2_star_solve[0]

m_1*(beta*(r + 1))**(1/rho)*(r + 1)/(r + (beta*(r + 1))**(1/rho) + 1)

In [9]:
# lambdify functions for c1 and c2
c1_func = sm.lambdify((m1,r,beta,rho),c1_star)
c2_func = sm.lambdify((m1,r,beta,rho),c2_star)

In [30]:
# define parameter values
m1 = 2
r = 0.03
beta = 0.95
rho = 2

In [43]:
# solve analytically and numerically

c1_s, c2_s = modelproject.solve_for_consumption(m1,r,beta,rho)

print(f'analytical solution is: c1 = {c1_func(m1,r,beta,rho)[0]:.3f}, c2 = {c2_func(m1,r,beta,rho)[0]:.3f}')
print(f'numerical solution is: c1 = {c1_s:.3f}, c2 = {c2_s:.3f}')
print(f'analytical and numerical solution are equal with numerical precision: c1: {np.isclose(c1_func(m1,r,beta,rho)[0],c1_s)} c2: {np.isclose(c2_func(m1,r,beta,rho)[0],c2_s)}')

analytical solution is: c1 = 1.020, c2 = 1.009
numerical solution is: c1 = 1.020, c2 = 1.009
analytical and numerical solution are equal with numerical precision: c1: True c2: True


Thus, the value function in **period 2** is given by:
$$
\begin{aligned}
v_{2}(m_{2}) &= \max_{c_{2}}\frac{c_{2}^{1-\rho}}{1-\rho}+(1-\tau)\nu\frac{(m_{2}-c_{2}+\kappa)^{1-\rho}}{1-\rho} \\
\text{s.t.} \\
c_{2} & \in [0,m_{2}]
\end{aligned}
$$

In **period 1**, the household solves 

$$
\begin{aligned}
v_{1}(m_{1}) &= \max_{c_{1}}\frac{c_{1}^{1-\rho}}{1-\rho}+\beta{v_2}(m_2)\\&\text{s.t.}&\\
m_2 &= (1+r)(m_{1}-c_{1})+y_{2} \\
c_{1} &\in [0,m_{1}]\\
\end{aligned}
$$